In [ ]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [ ]:
llm = ChatGroq(groq_api_key = os.getenv("GROQ_API_KEY"), model="Llama-3.3-70b-Versatile")

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

In [ ]:
class State(TypedDict):

    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [ ]:
graph_builder

In [ ]:
def chatbot(state:State):
    return {"messages":llm.invoke(state["messages"])}

In [ ]:
graph_builder.add_node("chatbot", chatbot)

In [ ]:
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)


In [ ]:
graph = graph_builder.compile()

In [ ]:
from IPython.display import Image, display

try:
    display(Image(graph.get_graph().draw_mermaid_png()))
except Exception:
    pass

In [ ]:
while True:
    user_input = input("USER: ")
    if user_input.lower() in ["quit", "q"]:
        print("Good Bye")
        break
    for event in graph.stream({"messages": ("user", user_input)}):
        print(event.values())
        for value in event.values():
            print(value["messages"])
            print("Assistant:", value["messages"].content)